In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

In [2]:
total_df = pd.DataFrame()
payment_df = pd.read_csv('./payment_window.csv', encoding='CP949')
bra_df = pd.read_csv('./bra_size_window.csv', encoding='CP949')
user_df = pd.read_csv('./user_window.csv', encoding='CP949')
pay_bra_df = pd.merge(payment_df, bra_df, how='outer', on='user_idx')
total_df = pd.merge(pay_bra_df, user_df, on='user_idx')
drop_cols = ['Unnamed: 0_x', 'Unnamed: 0', 'Unnamed: 0_y']
total_df.drop(drop_cols, axis=1, inplace=True)
total_df.reset_index(drop=True, inplace=True)

total_df['paid_date'] = pd.to_datetime(total_df['paid_date'])
total_df['joined_date'] = pd.to_datetime(total_df['joined_date'])
total_df['created_date'] = pd.to_datetime(total_df['created_date'])

In [3]:
total_df.head()

,user_idx,subscription_type,cart_discount_price,order_shipping_price,order_total_price,paid_date,size,created_date,status,third_party_join_type,joined_date
0,0973f573635c1b55a58f1d221f992bfeb0b3e007a93efb...,구독,5100.0,3000.0,12900.0,2020-10-21,80A,2020-09-25 20:30:29+09:00,0,NaN,2020-09-21 16:07:34+09:00
1,305181c0babb5077b46ccb298a41469729a6760b19e44d...,구독,5100.0,3000.0,12900.0,2020-10-09,75B,2020-10-21 22:49:35+09:00,0,NaN,2020-10-09 23:38:44+09:00
2,5fa450bdb2e831617f0132293d63638e0f93f1f62af4ca...,구독,5100.0,3000.0,12900.0,2020-10-11,80C,2020-10-21 05:11:29+09:00,0,NaN,2020-10-11 13:20:13+09:00
3,85649b91d8ac8d2e84faa6c54032254356fe02c69d5fa0...,구독,5100.0,3000.0,12900.0,2020-10-14,80B,2020-10-21 05:11:29+09:00,0,NaN,2020-10-14 22:35:15+09:00
4,05afd1abc70fce9afaa18482a0bb9c6cb082e65b1e0ab4...,구독,5100.0,3000.0,12900.0,2020-10-21,70A,2020-10-24 01:05:30+09:00,0,NaN,2020-10-21 19:38:04+09:00


In [9]:
total_df.shape

(17199, 11)

In [10]:
total_df.describe()

,cart_discount_price,order_shipping_price,order_total_price,status
count,7308.000000,7308.000000,7308.000000,17199.000000
mean,3267.939245,2970.443350,14658.949097,0.008838
std,2729.677090,296.324772,3071.579009,0.093596
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,3000.000000,12900.000000,0.000000
50%,5100.000000,3000.000000,12900.000000,0.000000
75%,5100.000000,3000.000000,18000.000000,0.000000
max,15000.000000,3000.000000,27900.000000,1.000000


In [11]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17199 entries, 0 to 17198
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype                                
---  ------                 --------------  -----                                
 0   user_idx               17199 non-null  object                               
 1   subscription_type      7277 non-null   object                               
 2   cart_discount_price    7308 non-null   float64                              
 3   order_shipping_price   7308 non-null   float64                              
 4   order_total_price      7308 non-null   float64                              
 5   paid_date              7308 non-null   datetime64[ns]                       
 6   size                   16588 non-null  object                               
 7   created_date           16602 non-null  datetime64[ns, pytz.FixedOffset(540)]
 8   status                 17199 non-null  int64                      

# 전처리

## 컬럼별 전처리

In [7]:
total_df.isnull().sum()

user_idx                     0
subscription_type         9922
cart_discount_price       9891
order_shipping_price      9891
order_total_price         9891
paid_date                 9891
size                       611
created_date               597
status                       0
third_party_join_type    15807
joined_date                  0
dtype: int64

In [14]:
total_df['subscription_type'] = total_df['subscription_type'].fillna('미구독')

In [15]:
total_df['cart_discount_price'].value_counts()

5100.0     4471
0.0        2765
15000.0      72
Name: cart_discount_price, dtype: int64

In [8]:
total_df['subscription_type'].value_counts()

구독    7277
Name: subscription_type, dtype: int64